In [1]:
import tensorflow as tf

from matchers import constant, dataset, metrics, utils

In [2]:
MAX_NAME_LENGTH = 30

### Load data

In [3]:
input_names, relevant_names, all_candidates = dataset.load_preprocess()

### Build token index mappings

In [4]:
char_to_idx_map, idx_to_char_map = utils.build_token_idx_maps()

### Convert names to ids

In [5]:
X_all_candidates = utils.convert_names_to_ids(all_candidates, char_to_idx_map, MAX_NAME_LENGTH)
X_all_candidates = utils.one_hot_encode(X_all_candidates, constant.VOCAB_SIZE + 1)

### Model

In [6]:
hidden_dim = 100

# Encoder
encoder_input = tf.keras.layers.Input(shape=(MAX_NAME_LENGTH, constant.VOCAB_SIZE + 1))
encoder_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim, 
                                                                    return_sequences=False), 
                                               name='encoder')(encoder_input)
# Decoder
h = tf.keras.layers.RepeatVector(MAX_NAME_LENGTH)(encoder_output)
h = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)(h)
decoder_output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(constant.VOCAB_SIZE + 1, 
                                                                       activation='softmax'))(h)

# Model
autoencoder = tf.keras.models.Model(encoder_input, decoder_output)
autoencoder.compile(loss='categorical_crossentropy', optimizer='adam')

# Model for just the encoder
# Used after the autoencoder is fully trained
encoder_model = tf.keras.models.Model(inputs=autoencoder.inputs, 
                                      outputs=autoencoder.get_layer('encoder').output)

In [7]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 29)]          0         
_________________________________________________________________
encoder (Bidirectional)      (None, 200)               104000    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 30, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 100)           120400    
_________________________________________________________________
time_distributed (TimeDistri (None, 30, 29)            2929      
Total params: 227,329
Trainable params: 227,329
Non-trainable params: 0
_________________________________________________________________


In [8]:
autoencoder.fit(X_all_candidates, 
                X_all_candidates, 
                epochs=100,
                batch_size=512)

Epoch 1/100
48/48 [==============================] - 19s 326ms/step - loss: 2.0195
Epoch 2/100
48/48 [==============================] - 19s 399ms/step - loss: 1.0181
Epoch 3/100
48/48 [==============================] - 20s 413ms/step - loss: 0.9040
Epoch 4/100
48/48 [==============================] - 21s 431ms/step - loss: 0.8523
Epoch 5/100
48/48 [==============================] - 18s 371ms/step - loss: 0.7779
Epoch 6/100
48/48 [==============================] - 18s 377ms/step - loss: 0.7414
Epoch 7/100
48/48 [==============================] - 18s 368ms/step - loss: 0.7707
Epoch 8/100
48/48 [==============================] - 17s 360ms/step - loss: 0.6823
Epoch 9/100
48/48 [==============================] - 18s 383ms/step - loss: 0.6427
Epoch 10/100
48/48 [==============================] - 17s 360ms/step - loss: 0.6640
Epoch 11/100
48/48 [==============================] - 18s 381ms/step - loss: 0.6230
Epoch 12/100
48/48 [==============================] - 18s 370ms/step - loss: 0.5911
E

48/48 [==============================] - 16s 327ms/step - loss: 0.0370
Epoch 99/100
48/48 [==============================] - 16s 324ms/step - loss: 0.0359
Epoch 100/100
48/48 [==============================] - 16s 335ms/step - loss: 0.0339


### Evaluation

In [9]:
X_input_names = utils.convert_names_to_ids(input_names, char_to_idx_map, MAX_NAME_LENGTH)
X_input_names = utils.one_hot_encode(X_input_names, constant.VOCAB_SIZE + 1)

In [10]:
X_input_names_encoded = encoder_model.predict(X_input_names)
X_candidates_encoded = encoder_model.predict(X_all_candidates)

In [11]:
candidates = utils.get_candidates_batch(X_input_names_encoded, 
                                        X_candidates_encoded, 
                                        all_candidates,
                                        num_candidates=10)

In [12]:
candidate_names = candidates[:, :, 0]

### mAP @ 1

In [13]:
metrics.mean_avg_precision_k(relevant_names, candidate_names, 1)

0.5117225590367576

### mAP @ 3

In [14]:
metrics.mean_avg_precision_k(relevant_names, candidate_names, 3)

0.4140840293381946

### Demo

In [15]:
test_name = ['<schumacher>']
test_name_sequence = utils.convert_names_to_ids(test_name, char_to_idx_map, MAX_NAME_LENGTH)
test_name_one_hot = utils.one_hot_encode(test_name_sequence, constant.VOCAB_SIZE + 1)
test_name_embedding = encoder_model.predict(test_name_one_hot)

utils.get_candidates_batch(test_name_embedding, 
                           X_candidates_encoded,
                           all_candidates,
                           num_candidates=10)

array([[['<schumacker>', 0.9707528352737427],
        ['<stelmacher>', 0.9584547877311707],
        ['<schumaker>', 0.9551969766616821],
        ['<schurhamer>', 0.9541249871253967],
        ['<stillmacher>', 0.9501252770423889],
        ['<schacher>', 0.9501103162765503],
        ['<schmelcher>', 0.9477940797805786],
        ['<schumpert>', 0.9368616342544556],
        ['<schmicker>', 0.9283653497695923],
        ['<standacher>', 0.9281593561172485]]], dtype=object)